# T5. REFERENCE LEVELS

In this notebook, we extract the reference levels from the t1_semantic segmentation/ t2_instance segmentation

## LIBRARIES

In [60]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

#import utils
from context import utils
import utils as utl
import utils.t1_utils as t1


In [61]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
%autoreload 2

## INPUTS

In [152]:
name='e003_labels'

path=Path(os.getcwd()).parents[2]/'data'
pcd_input_path=os.path.join(path,f'{name}.laz')
class_file=path/'_classes.json'

name=name.split('_')[0]
json_output_path=os.path.join(path,f'{name}_levels.json') 
geometry_output_path= os.path.join(path,f'{name}_levels.obj') # these are the bounding surfaces of the reference levels (optional)


Import Classes

In [64]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import PCD

In [69]:
pcdNodes=[]
#split pcd per object
for i in np.unique(laz['classes']):
    idx=np.where(laz['classes']==i)
    points=laz.xyz[idx]
    # colors=np.array([laz.red[idx],laz.green[idx],laz.blue[idx]])
    object_labels=laz['objects'][idx]

    class_obj=next((class_obj for class_obj in json_data['classes'] if float(class_obj['id']) ==i), json_data['classes'][0])
    class_name=class_obj['name']

    # pcd.colors=o3d.utility.Vector3dVector(colors)
    for j in np.unique(object_labels):
        
        new_points=points[np.where(object_labels==j)]
        if new_points.shape[0]>100:
            pcd=o3d.geometry.PointCloud()
            pcd.points=o3d.utility.Vector3dVector(new_points)

            pcdNodes.append(PointCloudNode(resource=pcd,
                                        class_id=i,
                                        object_id=j,
                                        color=ut.random_color(),
                                            name=class_name+f'_{j}'))

print(f'{len(pcdNodes)} pcdNodes created!')

48 pcdNodes created!


In [70]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd])

## PROCESSING

Group floor point clouds

In [86]:
#retrieve floor points
floorNodes=[n for n in pcdNodes if n.class_id ==0]
print(f'{len(floorNodes)} floorNodes detected!')
for n in floorNodes:
    print(f' {n.name} with {n.orientedBoundingBox}')


5 floorNodes detected!
 Floors_1  with OrientedBoundingBox: center: (5.84162, 1.7628, -1.50459), extent: 11.4926, 5.16798, 0.0574506)
 Floors_2  with OrientedBoundingBox: center: (-3.22753, -5.63086, -1.54735), extent: 5.24738, 2.18948, 0.0974616)
 Floors_3  with OrientedBoundingBox: center: (-0.815858, -6.50324, -1.64959), extent: 3.02764, 1.11434, 0.119771)
 Floors_4  with OrientedBoundingBox: center: (3.98498, 10.3573, -1.50599), extent: 13.8513, 12.9893, 0.0711255)
 Floors_5  with OrientedBoundingBox: center: (-1.98589, 0.503929, -1.46873), extent: 10.4484, 2.44964, 0.0977315)


In [91]:
# for now, put all floorNodes in a single group (1 reference level)
groupedFloorNodes=[floorNodes]
print(groupedFloorNodes)


[[<geomapi.nodes.pointcloudnode.PointCloudNode object at 0x0000021A87E85AB0>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x0000021A8B090640>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x0000021A88886D70>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x0000021A88885C60>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x0000021A88887010>]]


Compute reference Planes

In [139]:
#compute height underneight all floor points of a level (according to the lowest slab)
referenceNodes= []

for i,nodes in enumerate(groupedFloorNodes):
    #create sessionNode
    referenceNode=SessionNode(linkedNodes=nodes,
                                name=str(i)+'0')
    
    #determine height -> note that this can be negative
    weights=[float(len(np.asarray(n.resource.points))) for n in nodes]
    heights= [float(n.cartesianTransform[2,3]) for n in nodes]
    weighted_height= np.average(heights, weights=weights)

    #compute plane from cornerpoints orientedbounding box
    vertices=np.array([np.hstack((referenceNode.orientedBounds[0][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[1][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[2][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[4][0:2],weighted_height))])#,
    vertices=o3d.cpu.pybind.utility.Vector3dVector(vertices)
    triangles=o3d.cpu.pybind.utility.Vector3iVector(np.array([[0,1,2],[2,1,3]]))
    plane=o3d.geometry.TriangleMesh(vertices,triangles)

    #assign information to referenceNode
    referenceNode.plane=plane
    referenceNode.height=weighted_height
    
    referenceNodes.append(referenceNode)
    print(f' Level {referenceNode.name} created at height {referenceNode.height}')
# print(f'{len(referenceNodes)} referenceNodes created!')

 Level 00 created at height -1.510533772929771


In [136]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd,referenceNodes[0].plane])

In [107]:
{key:value for key, value in referenceNode.__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_linkedNodes': [<geomapi.nodes.pointcloudnode.PointCloudNode at 0x21a87e85ab0>,
 '_linkedSubjects': [rdflib.term.URIRef('file:///Floors_1'),
  rdflib.term.URIRef('file:///Floors_2'),
  rdflib.term.URIRef('file:///Floors_3'),
  rdflib.term.URIRef('file:///Floors_4'),
  rdflib.term.URIRef('file:///Floors_5')],
 '_subject': rdflib.term.URIRef('file:///a3e78300-eaa5-11ee-9be4-a86daaa4a627'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': '00',
 '_cartesianBounds': array([-5.77175325, 12.37057903, -8.33183713, 18.43981594, -1.789631  ,
        -1.30798256]),
 '_orientedBounds': array([[  6.74585019, -12.42514678,  -1.8615349 ],
        [ 16.43704216,  13.27820569,  -1.71713809],
        [-10.22306544,  -6.02798653,  -1.71918955],
        [  6.74784872, -12.42794525,  -1.49752447],
        [ -0.52987494,  19.67256747,  -1.21078231],
        [-10.22106691,  -6.03078501,  -1.35517912],
        [ 16.43904069,  13.27540722,  -1.35312766],
        [ -0.53187347,  19.67536594,  

## EXPORT

geometry

In [143]:
geometry_output_path

'c:\\Users\\Maarten\\OneDrive - KU Leuven\\2024-05 CVPR scan-to-BIM challenge\\data\\e003_labels_labels.laz'

In [142]:
joined_references=gmu.join_geometries([n.resource for n in referenceNodes])
success=o3d.io.write_triangle_mesh(filename=geometry_output_path, mesh=joined_references)
print(f' Saving joint references : {success}')
    

[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
 Saving joint references : False


json with reference heights

In [ ]:
#declare json
json_data = {
        "filename": ut.get_filename(geometry_output_path),
        "objects": []
    }

In [ ]:
#fill json
for n in referenceNodes:
    obj = {
            "name": n.name,
            "object_id":n.object_id,
            "centroid": {
                "x": n.center[0],
                "y": n.center[1],
                "z": n.center[2]
            },
            "dimensions": {
                "length": n.dimensions[0],
                "width": n.dimensions[1],
                "height": n.dimensions[2]
            },
            "rotations": {
                "x": 0,
                "y": 0,
                "z": n.rotations[0]
            }
            }
    json_data["objects"].append(obj)

In [ ]:
#write this information to the 3D detection json
with open(json_output_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)
print("JSON data written to file:", json_output_path)